# Loading packages

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Instantiate key constants & paths 

In [ ]:
MAIN_DIR = Path("../")
DATA_DIR = "data"
RAW_DIR = "raw"
STRUCTURED_DIR = "structured"
SEPARATOR = ";"
ENCODER = "utf-8"
RAW_SURNAME_FILE_NAME = "dpt2021.csv"
STRUCTURED_SURNAME_FILE_NAME = "SURNAME_2021.parquet"

RAW_SURNAME_FILE_PATH = MAIN_DIR / DATA_DIR / RAW_DIR / RAW_SURNAME_FILE_NAME
STRUCTURED_SURNAME_FILE_PATH = MAIN_DIR / DATA_DIR / STRUCTURED_DIR / STRUCTURED_SURNAME_FILE_NAME

# Loading raw file

In [ ]:
original_french_surname = pd.read_csv(filepath_or_buffer=RAW_SURNAME_FILE_PATH, sep=SEPARATOR)
original_french_surname.head()

# Renaming columns 

In [ ]:
original_french_surname = original_french_surname.rename(
    columns={
        "sexe": "GENDER",
        "preusuel": "SURNAME",
        "annais": "YEAR_OF_BIRTH",
        "dpt": "DEPARTMENT",
        "nombre": "NUMBER",
    }
)

original_french_surname.head()

# Memory Usage

In [ ]:
french_surname = original_french_surname.copy()

In [ ]:
# The + symbol indicates that the true memory usage could be higher, because pandas does not count the memory used by values in columns with dtype=object
french_surname.info()

In [ ]:
# Passing memory_usage="deep" will enable a more accurate memory usage report, accounting for the full usage of the contained objects.
# This is optional as it can be expensive to do this deeper introspection.
french_surname.info(memory_usage="deep")

# Improve Data Types

In [ ]:
french_surname.head()

## Gender

In [ ]:
french_surname["GENDER"].unique()

In [ ]:
print(np.iinfo(np.int8))
print(np.iinfo(np.int16))
print(np.iinfo(np.int32))
print(np.iinfo(np.int64))

In [ ]:
french_surname["GENDER"] = french_surname["GENDER"].astype(pd.Int8Dtype())

In [ ]:
# From 732MB to 707MB
french_surname.info(memory_usage="deep")

## Number

In [ ]:
french_surname["NUMBER"].min(), french_surname["NUMBER"].max() 

In [ ]:
# french_surname["NUMBER"] = french_surname["NUMBER"].astype("int8[pyarrow]")
french_surname["NUMBER"] = french_surname["NUMBER"].astype(pd.Int16Dtype())

In [ ]:
# From 707MB to 685MB 
french_surname.info(memory_usage="deep")

## Surname

- A common heuristic is to look at the percentage of unique values compared to the total number of rows. 
- For example, if less than 10% of the rows are unique, the column might be considered low cardinality. 
- Conversely, if most of the values are unique (e.g., more than 50%), the column is likely high cardinality. 
- These percentages are not hard rules but starting points for consideration.

In [ ]:
nunique_surname = french_surname["SURNAME"].nunique()
dataframe_length = len(french_surname)
percentage_of_unique_values = nunique_surname / dataframe_length
print(f"Number of unique surname : {nunique_surname}")
print(f"Dataframe length : {dataframe_length}")
print("Percentage of unique values : {:.3%}".format(percentage_of_unique_values))

In [ ]:
french_surname["SURNAME"] = french_surname["SURNAME"].astype(pd.CategoricalDtype())

In [ ]:
# From 685MB to 462MB 
french_surname.info(memory_usage="deep")

## Department

In [ ]:
nunique_dpt = french_surname["DEPARTMENT"].nunique()
dataframe_length = len(french_surname)
percentage_of_unique_values = nunique_dpt / dataframe_length
print(f"Number of unique deparment : {nunique_dpt}")
print(f"Dataframe length : {dataframe_length}")
print("Percentage of unique values : {:.3%}".format(percentage_of_unique_values))

In [ ]:
french_surname["DEPARTMENT"] = french_surname["DEPARTMENT"].astype(pd.CategoricalDtype())

In [ ]:
# From 462MB to 252MB 
french_surname.info(memory_usage="deep")

## Year of birth

In [ ]:
french_surname["YEAR_OF_BIRTH"].min(), french_surname["YEAR_OF_BIRTH"].max() 

In [ ]:
french_surname["YEAR_OF_BIRTH"].unique()

In [ ]:
# french_surname["YEAR_OF_BIRTH"] = french_surname["YEAR_OF_BIRTH"].astype("int16[pyarrow]")
# french_surname["YEAR_OF_BIRTH"] = french_surname["YEAR_OF_BIRTH"].replace({"XXXX": np.nan}).astype("int16[pyarrow]")
french_surname["YEAR_OF_BIRTH"] = french_surname["YEAR_OF_BIRTH"].astype(pd.StringDtype())

In [ ]:
# From 252MB to 36MB
french_surname.info(memory_usage="deep")

# 5. Loading data knowing data types

Une fois que vous avez déterminé les types de vos colonnes, il est important de les intégrer dès le chargement de vos données, afin de contourner l'inférence faite par pandas.

In [ ]:
# Loading data with appropriate data types
typed_french_surname = pd.read_csv(
    filepath_or_buffer=RAW_SURNAME_FILE_PATH,
    sep=SEPARATOR,
    encoding=ENCODER,
    dtype={
        "sexe": pd.Int8Dtype(),
        "preusuel": pd.CategoricalDtype(),
        "annais": pd.StringDtype(),  # we will replace 'XXXX' by NaN and then type as pd.Int16Dtype()
        "dpt": pd.CategoricalDtype(),
        "nombre": pd.Int16Dtype(),
    },
)

# Rename columns
structured_french_surname = typed_french_surname.rename(
    columns={
        "sexe": "GENDER",
        "preusuel": "SURNAME",
        "annais": "YEAR_OF_BIRTH",
        "dpt": "DEPARTMENT",
        "nombre": "NUMBER",
    }
)

structured_french_surname.info(memory_usage="deep")

Note that steps within cell above may be considered as the mininal `raw to structured` process (from raw untyped file to structured typed file).

# Saving structured file to parquet

In [ ]:
# To save data types
structured_french_surname.to_parquet(path=STRUCTURED_SURNAME_FILE_PATH)

# Loading structured file

In [ ]:
structured_french_surname = pd.read_parquet(path=STRUCTURED_SURNAME_FILE_PATH)
structured_french_surname.info(memory_usage="deep")

In [ ]:
structured_french_surname.head()

# Mutation VS Chaining

Task #1 : Rename `GENDER` as `GENDER_CODE`, Create `GENDER_LABEL` based on `GENDER_CODE` values, Type `GENDER_LABEL` as category 

Task #2 : I want to create a column named `NUNIQUE_NUMBER_BY_DEPARTMENT` counting the number of unique surname by department

Task #3 : rename categories "XXXX" to "Unknown" in `YEAR_OF_BIRTH` and "XX" by "Unkwown" in `DEPARTMENT`

## Mutation - "Bad" practice (pros and cons discussed later)

In [ ]:
structured_french_surname_mutation = structured_french_surname.copy()

# Task # 1
structured_french_surname_mutation = structured_french_surname_mutation.rename(columns={"GENDER": "GENDER_CODE"}) 
structured_french_surname_mutation["GENDER_LABEL"] = structured_french_surname_mutation["GENDER_CODE"].map({1: "MALE", 2: "FEMALE"})
structured_french_surname_mutation["GENDER_LABEL"] = structured_french_surname_mutation["GENDER_LABEL"].astype("category")

# Task # 2
structured_french_surname_mutation["NUNIQUE_NUMBER_BY_DEPARTMENT"] = structured_french_surname_mutation.groupby(by="DEPARTMENT", observed=True)["NUMBER"].transform("nunique")
structured_french_surname_mutation["NUNIQUE_NUMBER_BY_DEPARTMENT"] = structured_french_surname_mutation["NUNIQUE_NUMBER_BY_DEPARTMENT"].astype(pd.Int16Dtype())

# Task # 3
structured_french_surname_mutation["YEAR_OF_BIRTH"] = structured_french_surname_mutation["YEAR_OF_BIRTH"].replace({'XXXX': np.nan})
structured_french_surname_mutation["YEAR_OF_BIRTH"] = structured_french_surname_mutation["YEAR_OF_BIRTH"].astype(pd.Int16Dtype())
structured_french_surname_mutation["DEPARTMENT"] = structured_french_surname_mutation["DEPARTMENT"].cat.rename_categories({"XX": "Unknown"})

# Task # 4
mask_gender = structured_french_surname_mutation["GENDER_LABEL"] == "FEMALE"
mask_year_of_birth = structured_french_surname_mutation["YEAR_OF_BIRTH"].between(1980, 1990)
structured_french_surname_mutation = structured_french_surname_mutation[mask_gender & mask_year_of_birth]

# Task # 5
structured_french_surname_mutation = structured_french_surname_mutation.reset_index(drop=True)

structured_french_surname_mutation.head()

## Chaining - "Best" practice (pros and cons discussed later)

- Plus complexe car nécessite de mieux planifier ses tâches, nécessite une réelle planification avant exécution

Comparation avec une recette de cuisine (on ne raisonne non pas objet par objet mais par type de tâche)
- Je vais pas prendre la farine, la préparer, la cuire, puis les oeufs, les battre et la cuire, etc.
- C'est pas logique de faire ingrédient par ingrédient, il y a une logique d'ensemble à suivre
- On raisonne par type de tâche (ex: la pesée des ingrédients) et non plus par colonne (ex: la farine)

- La différence majeure est la façon de trier les informations - la 2ème méthode permet de + facilement retrouver les informations
- Ex livre d'une bibliothèque : trouver le moyen le + logique de trier les opérations et de pouvoir les retrouver aisément (Nom ou genre = OK, date d'édition = POK)

In [ ]:
structured_french_surname_chaining = (structured_french_surname
    .rename(columns={"GENDER": "GENDER_CODE"})
    .assign(
        GENDER_LABEL=lambda df_: df_["GENDER_CODE"].map({1: "MALE", 2: "FEMALE"}),
        NUNIQUE_NUMBER_BY_DEPARTMENT=structured_french_surname.groupby(by="DEPARTMENT", observed=True)["NUMBER"].transform("nunique"),
        YEAR_OF_BIRTH=structured_french_surname["YEAR_OF_BIRTH"].replace({'XXXX': np.nan}),
        DEPARTMENT=structured_french_surname["DEPARTMENT"].cat.rename_categories({"XX": "Unknown"}),
    )
    .astype({
        "GENDER_LABEL": pd.CategoricalDtype(),
        "NUNIQUE_NUMBER_BY_DEPARTMENT": pd.Int16Dtype(),
        "YEAR_OF_BIRTH": pd.Int16Dtype()},
    )
    .query("1980 <= YEAR_OF_BIRTH <= 1990 and GENDER_LABEL == 'FEMALE'")
    .reset_index(drop=True)
)

structured_french_surname_chaining.head()

In [ ]:
structured_french_surname_mutation.info(memory_usage="deep")

In [ ]:
structured_french_surname_chaining.info(memory_usage="deep")

- Chaining transforms a DataFrame according to a multi-step procedure all at once. 
- This guarantees the full and proper application of each pandas method, thus mitigating the risk of bugs. 
- The code is more readable with each line cleanly representing a distinct operation 
- (note: many Python code formatters will destroy this structure - wrap your pandas code blocks with `#fmt: off` and `#fmt: on` to prevent this). 
- Chaining will feel natural for R users familiar with the `magrittr %>%` operator.

# Debugging

## Simplest way of debugging : Comment each line one by one

In [ ]:
# First way to debbug : Comment each line one by one to visualise intermediate results
(structured_french_surname
    .rename(columns={"GENDER": "GENDER_CODE"})
    .assign(
        GENDER_LABEL=lambda df_: df_["GENDER_CODE"].map({1: "MALE", 2: "FEMALE"}),
        # NUNIQUE_NUMBER_BY_DEPARTMENT=structured_french_surname.groupby(by="DEPARTMENT", observed=True)["NUMBER"].transform("nunique"),
        # YEAR_OF_BIRTH=structured_french_surname["YEAR_OF_BIRTH"].replace({'XXXX': np.nan}),
        # DEPARTMENT=structured_french_surname["DEPARTMENT"].cat.rename_categories({"XX": "Unknown"}),
    )
    # .astype({
    #     "GENDER_LABEL": pd.CategoricalDtype(),
    #     "NUNIQUE_NUMBER_BY_DEPARTMENT": pd.Int16Dtype(),
    #     "YEAR_OF_BIRTH": pd.Int16Dtype(),
    # })
    # .query("1980 <= YEAR_OF_BIRTH <= 1990 and GENDER_LABEL == 'FEMALE'")
    # .reset_index(drop=True)
)

## Advanced way of debugging : create intermediate displays and/or dataframes (a notebook-friendly alternative to debug mode)

In [ ]:
def display_five_first_rows(df_):
    print("display five first rows :")
    display(df_.head())
    return df_

def display_five_first_female(df_):
    print("display five first female :")
    display(df_.query("GENDER_LABEL == 'FEMALE'").head())
    return df_

def get_dataframe(df_, var_name):
    globals()[var_name] = df_
    return df_

In [ ]:
structured_french_surname_chaining = (structured_french_surname
    .rename(columns={"GENDER": "GENDER_CODE"})
    .assign(
        GENDER_LABEL=lambda df_: df_["GENDER_CODE"].map({1: "MALE", 2: "FEMALE"}),
        NUNIQUE_NUMBER_BY_DEPARTMENT=structured_french_surname.groupby(by="DEPARTMENT", observed=True)["NUMBER"].transform("nunique"),
        YEAR_OF_BIRTH=structured_french_surname["YEAR_OF_BIRTH"].replace({'XXXX': np.nan}),
        DEPARTMENT=structured_french_surname["DEPARTMENT"].cat.rename_categories({"XX": "Unknown"}),
    )
    .pipe(display_five_first_rows)
    .pipe(get_dataframe, "after_assign_before_astype")
    .astype({
        "GENDER_LABEL": pd.CategoricalDtype(),
        "NUNIQUE_NUMBER_BY_DEPARTMENT": pd.Int16Dtype(),
        "YEAR_OF_BIRTH": pd.Int16Dtype(),
    })
    .pipe(display_five_first_female)
    .pipe(get_dataframe, "after_astype_before_query")
    .query("1980 <= YEAR_OF_BIRTH <= 1990 and GENDER_LABEL == 'FEMALE'")
    .reset_index(drop=True)
    .pipe(get_dataframe, "final_df")
)

In [ ]:
after_assign_before_astype.info(memory_usage="deep")

In [ ]:
after_astype_before_query.info(memory_usage="deep")

In [ ]:
final_df.info(memory_usage="deep")

# Limit of chaining (personal opinion) & Style

- The longer the chain is, the harder it becomes to quickly understand the process
- Especially when whe reproduce a similar method
    - In this case : 2 times the `.assign()` method and two times the `.groupby()` method

In [ ]:
def thousand_separator(number: int) -> str:
    return "{:,}".format(number).replace(",", " ")

In [ ]:
(structured_french_surname
    .rename(columns={"GENDER": "GENDER_CODE"})
    .assign(
        GENDER_LABEL=lambda df_: df_["GENDER_CODE"].map({1: "MALE", 2: "FEMALE"}),
        NUNIQUE_NUMBER_BY_DEPARTMENT=structured_french_surname.groupby(by="DEPARTMENT", observed=True)["NUMBER"].transform("nunique"),
        YEAR_OF_BIRTH=structured_french_surname["YEAR_OF_BIRTH"].replace({'XXXX': np.nan}),
        DEPARTMENT=structured_french_surname["DEPARTMENT"].cat.rename_categories({"XX": "Unknown"}),
    )
    .astype({
        "GENDER_LABEL": pd.CategoricalDtype(),
        "NUNIQUE_NUMBER_BY_DEPARTMENT": pd.Int16Dtype(),
        "YEAR_OF_BIRTH": pd.Int16Dtype()},
    )
    .query("YEAR_OF_BIRTH in [2017, 2018, 2019, 2020, 2021] and SURNAME != '_PRENOMS_RARES'")
    .groupby(by=['GENDER_LABEL', "SURNAME"], observed=True, as_index=False)
    .agg({"NUMBER": "sum"})
    .sort_values(by=["GENDER_LABEL", "NUMBER"], ascending=False)
    .groupby(['GENDER_LABEL'], observed=True).head(5)
    .assign(
        TOTAL_NUMBER = lambda df_: df_.groupby("GENDER_LABEL", observed=True)["NUMBER"].transform("sum"),
        PROPORTION = lambda df_: df_["NUMBER"] / df_["TOTAL_NUMBER"],
    )
    .reset_index(drop=True)
    .style.format({
        "NUMBER": thousand_separator, 
        "TOTAL_NUMBER": thousand_separator, 
        "PROPORTION": "{:.0%}"
    })
)

- That's why I recommend an intermediate step (a compromise between mutation and chaining)
- For example :
    - In the middle of the chain, or
    - Where it really makes sense in the context of the project
    - Personal rule of thumb : if I need to reuse a method, it's time to mutate

In [ ]:
surname_by_gender = (structured_french_surname
    .rename(columns={"GENDER": "GENDER_CODE"})
    .assign(
        GENDER_LABEL=lambda df_: df_["GENDER_CODE"].map({1: "MALE", 2: "FEMALE"}),
        NUNIQUE_NUMBER_BY_DEPARTMENT=structured_french_surname.groupby(by="DEPARTMENT", observed=True)["NUMBER"].transform("nunique"),
        YEAR_OF_BIRTH=structured_french_surname["YEAR_OF_BIRTH"].replace({'XXXX': np.nan}),
        DEPARTMENT=structured_french_surname["DEPARTMENT"].cat.rename_categories({"XX": "Unknown"}),
    )
    .astype({
        "GENDER_LABEL": pd.CategoricalDtype(),
        "NUNIQUE_NUMBER_BY_DEPARTMENT": pd.Int16Dtype(),
        "YEAR_OF_BIRTH": pd.Int16Dtype()},
    )
    .query("YEAR_OF_BIRTH in [2017, 2018, 2019, 2020, 2021] and SURNAME != '_PRENOMS_RARES'")
    .groupby(by=['GENDER_LABEL', "SURNAME"], observed=True, as_index=False)
    .agg({"NUMBER": "sum"})
    .sort_values(by=["GENDER_LABEL", "NUMBER"], ascending=False)
)

surname_by_gender

In [ ]:
(surname_by_gender
    .groupby(['GENDER_LABEL'], observed=True).head(5)
    .assign(
        TOTAL_NUMBER=lambda df_: df_.groupby("GENDER_LABEL", observed=True)["NUMBER"].transform("sum"),
        PROPORTION=lambda df_: df_["NUMBER"] / df_["TOTAL_NUMBER"],
    )
    .reset_index(drop=True)
    .style.format({"NUMBER": thousand_separator, "TOTAL_NUMBER": thousand_separator, "PROPORTION": "{:.0%}"})
)